In [5]:
pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import xml.etree.ElementTree as ET
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import RDF, RDFS, DCTERMS, XSD
import xml.dom.minidom as md
import datetime

In [9]:
def xml_to_ttl(xmlpath, ttlpath):
  #ファイルの読み取り
  with open(xmlpath, 'r', encoding = 'utf-8') as f:
    tree = ET.parse(f) 
  root = tree.getroot()

  #ttlデータの作成
  data = Graph()

  pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')
  data.bind('pd3', pd3)
  data.bind('rdf', RDF)
  data.bind('dcterms', DCTERMS)
  data.bind('rdfs', RDFS)
  epURI_Num = 0

  for diagram in root.iter('diagram'):
    epstyle = ''
    if(diagram[0][0][0].get('style') != None):
      epstyle = diagram[0][0][0].get('style').split(';')

    for element in epstyle:
      if('URI=' in element):

        #タイムスタンプをuriに追加
        t_delta = datetime.timedelta(hours=9)
        JST = datetime.timezone(t_delta, 'JST')
        now = datetime.datetime.now(JST)
        d = '{:%Y%m%d%H%M%S}'.format(now) 

        epuri = element.replace('URI=','')
        if(epuri[-1] == '/'):
          del epuri[-1]
        epuri = epuri + '/' + d + '/'
        epURI = Namespace(epuri)
        ep = URIRef(epuri)
        data.add((ep, RDF.type, pd3.EngineeringProcess))
      elif('prefix=' in element):
        prefix = element.replace('prefix=','')
        data.bind(prefix, epURI)
      elif('title=' in element):
        title = element.replace('title=', '')
        data.add((ep, DCTERMS.title, Literal(title)))
      elif('creator=' in element):
        creator = element.replace('creator=', '')
        data.add((ep, DCTERMS.creator, Literal(creator)))
      elif('description=' in element):
        description = element.replace('description=', '')
        data.add((ep, DCTERMS.description, Literal(description)))
      elif('identifier=' in element):
        identifier = element.replace('identifier=', '')
        data.add((ep, DCTERMS.identifier, Literal(identifier)))
      elif('epType=' in element):
        eptype = element.replace('eptype', '')
        data.add((ep, pd3.epType, Literal(eptype)))

    epURI_Num += 1  

    #エンティティの情報を入手
    for mxCell in diagram.iter('mxCell'):
      style = mxCell.get('style')
      if(style != None):
        #action
        if('pd3type=action' in style):
          #★★★★★★★
          # 2022/1/12 mod Start Man
          id = mxCell.get('id').replace('_','-')
          # 2022/1/12 mod End Man
          action = URIRef(epuri + id)
          #idとvalueを取得
          value = mxCell.get('value')

          data.add((action, RDF.type, pd3.Action))
          data.add((action, pd3.id, Literal(id)))
          data.add((action, pd3.value, Literal(value)))

          #座標、形状を取得
          data.add((action, pd3.geoBoundingWidth, Literal(mxCell[0].get('width'))))
          data.add((action, pd3.geoBoundingHeight, Literal(mxCell[0].get('height'))))
          data.add((action, pd3.geoBoundingX, Literal(mxCell[0].get('x'))))
          data.add((action, pd3.geoBoundingY, Literal(mxCell[0].get('y'))))

          pd3actioncheck = True
          for element in style.split(';'):
            if('pd3layer=' in element):
              layer = Literal(element.replace('pd3layer=', ''))
              data.add((action, pd3.layer, layer))
            elif('pd3action=' in element):
              actionType = element.replace('pd3action=', '')
              if(actionType == 'ECDP'):
                actionType = 'define problem'
              elif(actionType == 'ECCAI'):
                actionType = 'collect/analyze information'
              elif(actionType == 'ECGH'):
                actionType = 'generate hypothesis'
              elif(actionType == 'ECESI'):
                actionType = 'evaluate/select information'
              elif(actionType == 'ECEX'):
                actionType = 'execute'
              data.add((action, pd3.actionType, Literal(actionType)))
              pd3actioncheck = False
            elif('seeAlso=' in element):
              # 2022/1/17 mod Start Man
              seeEntities = element.replace('seeAlso=', '').split(',')
              for seeEntity in seeEntities:
                #seeAlso uriを取得
                seeTemp= seeEntity.replace('seeAlso=', '').split('[')
                print(seeTemp)
                if(seeTemp[0] != None) and (seeTemp[1] != None):
                  seeURI = Namespace(seeTemp[0])
                  #seeAlso prefixとidを取得
                  seeprefix = seeTemp[1].split(']')
                  if(seeprefix[0] != None) and (seeprefix[1] != None):
                    data.bind(seeprefix[0], seeURI)
                    data.add((action, RDFS.seeAlso, URIRef(seeURI + seeprefix[1])))
              """
              seeEntities = element.replace('seeAlso=', '').replace('['+prefix+']', '').split(',')
              for seeEntity in seeEntities:
                  data.add((action, RDFS.seeAlso, URIRef(epuri + seeEntity)))
              """
              # 2022/1/17 mod End Man
          if(pd3actioncheck):
            data.add((action,pd3.actionType, Literal('nil')))
          pd3actioncheck = True

          #attribution, container, input, outputを取得
          attribution_id = mxCell.get('parent')

          for mxCell1 in diagram.iter('mxCell'):
            if(mxCell1.get('target') == mxCell.get('id')):
              data.add((action, pd3.input, URIRef(epuri + mxCell1.get('id').replace('_','-'))))
              source_id = mxCell1.get('source')
              for mxCell2 in diagram.iter('mxCell'):
                if((mxCell2.get('style') != None) & (mxCell2.get('id') == source_id)):
                  if('pd3type=container' in mxCell2.get('style')):
                    data.add((action, pd3.expansion, URIRef(epuri + mxCell2.get('id').replace('_','-'))))
                    break
            elif((mxCell1.get('id') == attribution_id) & (mxCell1.get('style') != None)):
              data.add((action, pd3.attribution, URIRef(epuri + attribution_id.replace('_','-'))))
            elif(mxCell1.get('source') == mxCell.get('id')):
              data.add((action, pd3.output, URIRef(epuri + mxCell1.get('id').replace('_','-'))))

        #container
        elif('pd3type=container' in style):
          container = URIRef(epuri + mxCell.get('id').replace('_','-'))
          #idを取得
          id = mxCell.get('id')
          data.add((container, RDF.type, pd3.Container))
          data.add((container, pd3.id, Literal(mxCell.get('id').replace('_','-'))))
          
          #座標、形状を取得
          data.add((container, pd3.geoBoundingWidth, Literal(mxCell[0].get('width'))))
          data.add((container, pd3.geoBoundingHeight, Literal(mxCell[0].get('height'))))
          data.add((container, pd3.geoBoundingX, Literal(mxCell[0].get('x'))))
          data.add((container, pd3.geoBoundingY, Literal(mxCell[0].get('y'))))

          #layer, containerTypeを取得
          for element in style.split(';'):
            if('pd3layer=' in element):
              layer = Literal(element.replace('pd3layer=', ''))
              data.add((container, pd3.layer, layer))
            elif('containertype=' in element):
              containertype = element.replace('containertype=', '')
              data.add((container, pd3.containerType, Literal(containertype)))
            elif('seeAlso=' in element):
              seeEntities = element.replace('seeAlso=', '').replace('['+prefix+']', '').split(',')
              for seeEntity in seeEntities:
                data.add((action, RDFS.seeAlso, URIRef(epuri + seeEntity)))
              
          #valueを取得
          if(containertype == 'whilebox' or containertype == 'whilecontainer'):
            value = 'nil'
          else:
            value = mxCell.get('value')
          data.add((container, pd3.value, Literal(value)))

          #member, target, source, contractionを取得
          for mxCell1 in diagram.iter('mxCell'):
            if(mxCell1.get('parent') == id):
              member_id = mxCell1.get('id').replace('_','-')
              data.add((container, pd3.member, URIRef(epuri + member_id)))
            elif(mxCell1.get('source') == id):
              target_id = mxCell1.get('target')
              if target_id:
                data.add((container, pd3.output, URIRef(epuri + target_id.replace('_','-'))))
              for mxCell2 in diagram.iter('mxCell'):
                if((mxCell2.get('style') != None) & (mxCell2.get('id') == target_id)):
                  if('pd3type=action' in mxCell2.get('style')):
                    data.add((container, pd3.contraction, URIRef(epuri + mxCell2.get('id').replace('_','-'))))
                    break

        #arc
        elif('pd3type=arc' in style):
          if(mxCell.get('target') != None):
            arc = URIRef(epuri + mxCell.get('id').replace('_','-'))

            #id,valueを取得
            id = mxCell.get('id')
            value = mxCell.get('value')
            if(value == None or value == ''):
              value = ''
              for mxCell1 in diagram.iter('mxCell'):
                  if(mxCell1.get('style') != None):
                    if('edgeLabel' in mxCell1.get('style')):
                      if(id == mxCell1.get('parent')):
                        value = mxCell1.get('value')
                        break
            data.add((arc, pd3.id, Literal(mxCell.get('id').replace('_','-'))))
            data.add((arc, pd3.value, Literal(value)))

            #layerを取得
            for element in style.split(';'):
              if('pd3layer=' in element):
                layer = element.replace('pd3layer=', '')
            data.add((arc, pd3.layer, Literal(layer)))

            #attributionを取得
            attribution_id = mxCell.get('parent')
            for mxCell1 in diagram.iter('mxCell'):
              if((mxCell1.get('id') == attribution_id) & (mxCell1.get('style') != None)):
                data.add((arc, pd3.attribution, URIRef(epuri + attribution_id.replace('_','-'))))

            #sourceがあればFlowか, ContainerFlowか, ObjectをsourceとするSupFlow
            source = mxCell.get('source')
            target = mxCell.get('target')
            if(source != None):
              #source, targetを取得
              data.add((arc, pd3.source, URIRef(epuri + source.replace('_','-'))))
              data.add((arc, pd3.target, URIRef(epuri + target.replace('_','-'))))

              #exitX, exitY, entryX, entryYを取得            
              for element in style.split(';'):
                if('exitX=' in element):
                  exitX = element.replace('exitX=', '')
                  data.add((arc, pd3.exitX, Literal(exitX)))
                elif('exitY=' in element):
                  exitY = element.replace('exitY=', '')
                  data.add((arc, pd3.exitY, Literal(exitY)))
                elif('entryX=' in element):
                  entryX = element.replace('entryX=', '')
                  data.add((arc, pd3.entryX, Literal(entryX)))
                elif('entryY=' in element):
                  entryY = element.replace('entryY=', '')
                  data.add((arc, pd3.entryY, Literal(entryY)))
                elif('seeAlso=' in element):
                  seeEntities = element.replace('seeAlso=', '').replace('['+prefix+']', '').split(',')
                  for seeEntity in seeEntities:
                      data.add((action, RDFS.seeAlso, URIRef(epuri + seeEntity)))
                    
              for mxCell1 in diagram.iter('mxCell'):
                if(mxCell1.get('id') == source):
                  #Flow
                  if(mxCell1.get('style') != None):
                    if('pd3type=action' in mxCell1.get('style')):
                        data.add((arc, RDF.type, pd3.Flow))
                        data.add((arc, pd3.arcType, Literal('information')))
                        break

                      #ContainerFlow
                    elif('pd3type=container' in mxCell1.get('style')):
                      data.add((arc, RDF.type, pd3.ContainerFlow))
                      data.add((arc, pd3.arcType, Literal('hierarchization')))
                      break
                    #tool/knowledgeのSupFlow
                    elif('pd3type=object' in mxCell1.get('style')):
                      data.add((arc, RDF.type, pd3.SupFlow))
                      data.add((arc, pd3.arcType, Literal('tool/knowledge')))
                      break

            else:
              #sourceがなければSupFlow
              data.add((arc, RDF.type, pd3.SupFlow))

              #targetの取得
              data.add((arc, pd3.target, URIRef(epuri + target.replace('_','-'))))

              #entryX,entryYの取得
              for element in style.split(';'):
                if('entryX=' in element):
                  entryX = element.replace('entryX=', '')
                  data.add((arc, pd3.entryX, Literal(entryX)))
                elif('entryY=' in element):
                  entryY = element.replace('entryY=', '')
                  data.add((arc, pd3.entryY, Literal(entryY)))
                elif('seeAlso=' in element):
                  seeEntities = element.replace('seeAlso=', '').replace('['+prefix+']', '').split(',')
                  for seeEntity in seeEntities:
                      data.add((action, RDFS.seeAlso, URIRef(epuri + seeEntity)))
                      
              #arcTypeの取得
              if("entryY=1;" in style):
                data.add((arc, pd3.arcType, Literal('tool/knowledge')))
              elif("entryY=0.5;" in style):
                data.add((arc, pd3.arcType, Literal('rationale')))
              elif("entryY=0;" in style):
                if("entryX=0.5;" in style):
                  data.add((arc, pd3.arcType, Literal('intention')))
                elif("entryX=1;" in style):
                  data.add((arc, pd3.arcType, Literal('annotation')))

              #位置を取得
              for mxPoint in mxCell.iter('mxPoint'):
                if(mxPoint.get('as') == 'sourcePoint'):
                  data.add((arc, pd3.geoSourcePointX, Literal(mxPoint.get('x'))))
                  data.add((arc, pd3.geoSourcePointY, Literal(mxPoint.get('y'))))
                elif(mxPoint.get('as') == 'targetPoint'):
                  data.add((arc, pd3.geoTargetPointX, Literal(mxPoint.get('x'))))
                  data.add((arc, pd3.geoTargetPointY, Literal(mxPoint.get('y'))))
        # 2022/1/11 add Start Man
        #object
        elif('pd3type=object' in style):
          object = URIRef(epuri + mxCell.get('id').replace('_','-'))
          #idとvalueを取得
          id = mxCell.get('id').replace('_','-')
          value = mxCell.get('value')

          data.add((object, RDF.type, pd3.Object))
          data.add((object, pd3.id, Literal(id)))
          data.add((object, pd3.value, Literal(value)))

          #座標、形状を取得
          data.add((object, pd3.geoBoundingWidth, Literal(mxCell[0].get('width'))))
          data.add((object, pd3.geoBoundingHeight, Literal(mxCell[0].get('height'))))
          data.add((object, pd3.geoBoundingX, Literal(mxCell[0].get('x'))))
          data.add((object, pd3.geoBoundingY, Literal(mxCell[0].get('y'))))

          #layerを取得
          for element in style.split(';'):
            if('pd3layer=' in element):
              layer = element.replace('pd3layer=', '')
          data.add((object, pd3.layer, Literal(layer)))

          #outputを取得
          for mxCell1 in diagram.iter('mxCell'):
            if(mxCell1.get('source') == mxCell.get('id')):
              data.add((object, pd3.output, URIRef(epuri + (mxCell1.get('id').replace('_','-')))))

          #attributionを取得
          attribution_id = mxCell.get('parent')
          for mxCell1 in diagram.iter('mxCell'):
            if((mxCell1.get('id') == attribution_id) & (mxCell1.get('style') != None)):
              data.add((object, pd3.attribution, URIRef(epuri + attribution_id.replace('_','-'))))       
        # 2022/1/11 add End Man

    for UserObject in diagram.iter('UserObject'):
      id = UserObject.get('id')
      value = UserObject.get('label')
      object = URIRef(epuri + id)

      #idとvalueの取得
      data.add((object, RDF.type, pd3.Object))
      data.add((object, pd3.id, Literal(id)))
      data.add((object, pd3.value, Literal(value)))

      #外部参照のurlの取得
      # seeAlso_url = UserObject.get('link')
      # print(seeAlso_url)
      # data.add((object, RDFS.seeAlso, seeAlso_url))

      #レイヤーの取得
      style = UserObject[0].get('style').split(';')
      for element in style:
        if('pd3layer=' in element):
          layer = element.replace('pd3layer=', '')
          data.add((object, pd3.layer, Literal(layer)))

      #座標、形状を取得
      data.add((object, pd3.geoBoundingWidth, Literal(UserObject[0][0].get('width'))))
      data.add((object, pd3.geoBoundingHeight, Literal(UserObject[0][0].get('height'))))
      data.add((object, pd3.geoBoundingX, Literal(UserObject[0][0].get('x'))))
      data.add((object, pd3.geoBoundingY, Literal(UserObject[0][0].get('y'))))

  # print(data.serialize())
  with open(ttlpath, 'w', encoding = 'utf-8') as f:
    f.write(data.serialize())

In [ ]:
def ttl_to_xml(ttlpath, xmlpath):

  pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')

  # 2022/1/12 mod Start Man
  with open(ttlpath, 'r', encoding='utf-8') as f:
    g = Graph()
    g.parse(f)

  """
  g = Graph()
  g.parse(ttlpath)
  """
  # 2022/1/12 mod End Man

  mxfile = ET.Element('mxfile')
  diagram = ET.SubElement(mxfile, 'diagram')
  mxGraphModel = ET.SubElement(diagram, 'mxGraphModel')
  root = ET.SubElement(mxGraphModel, 'root')

  #EPのメタ情報
  mxCell = ET.SubElement(root, 'mxCell')
  mxCell.set('id', '0')
  title, creator, description, identifier = '', '', '', ''
  for s, p, o in g.triples((None, RDF.type, pd3.EngineeringProcess)):
    URI = str(s)
    epstyle = 'URI=' + URI + ';'
    for ns in g.namespaces():
      if(ns[1] == s):
        prefix = str(ns[0])
        epstyle += 'prefix=' + prefix + ';'
    for s2, p2, o2 in g.triples((s, None, None)):
      if(p2 == DCTERMS.title):
        title = str(o2)
        epstyle += 'title=' + title + ';'
      elif(p2 == DCTERMS.creator):
        creator = str(o2)
        epstyle += 'creator=' + creator + ';'
      elif(p2 == DCTERMS.description):
        description = str(o2)
        epstyle += 'description=' + description + ';'
      elif(p2 == DCTERMS.identifier):
        identifier = str(o2)
        epstyle += 'identifier=' + identifier + ';'
  mxCell.set('style', epstyle)

  #id=1のmxCell
  mxCell = ET.SubElement(root, 'mxCell')
  mxCell.set('id', '1')
  mxCell.set('parent', '0')

  for s,p,o in g.triples((None, RDF.type, None)):
    if(o != pd3.EngineeringProcess):
      mxCell = ET.SubElement(root, 'mxCell')
    #idとvalueを取得
    for s1,p1,o1 in g.triples((s, pd3.id, None)):
      id = str(o1)
      mxCell.set('id', id)  
    for s1, p1, o1 in g.triples((s, pd3.value, None)):
      value = str(o1)
      mxCell.set('value', value)
    #Action
    if(o == pd3.Action):
      style = "rounded=0;whiteSpace=wrap;html=1;pd3type=action;fontSize=14;"
      for s2, p2, o2 in g.triples((s, None, None)):
        if(p2 == pd3.layer):
          layer = str(o2)
        elif(p2 == pd3.actionType):
          actionType = str(o2)
      if(layer == 'phys'):
        if(actionType == 'start' or actionType == 'end'):
          style += 'dashed=1;'+'pd3action='+actionType+';'
        style += 'pd3layer=phys;fillColor=#d5e8d4;strokeColor=#82b366;'
      elif(layer == 'info'):
        if(actionType == 'start' or actionType == 'end'):
          style += 'dashed=1;'+'pd3action='+actionType+';'
        style += 'pd3layer=info;fillColor=#dae8fc;strokeColor=#6c8ebf;'
      elif(layer == 'topic'):
        style += 'pd3layer=topic;'
        if(actionType == 'start' or actionType == 'end'):
          style += 'dashed=1;'+'pd3action='+actionType+';'
          style += 'fillColor=#ffe6cc;strokeColor=#d79b00;'
        elif(actionType == 'nil'):
          style += 'fillColor=#ffe6cc;strokeColor=#d79b00;'
        elif(actionType == 'define problem'):
          style += "pd3action=ECDP;strokeColor=#d79b00;fontColor=#ffffff;fillColor=#D1BC35;"
        elif(actionType == 'collect/analyze information'):
          style += "pd3action=ECCAI;strokeColor=#d79b00;fontColor=#ffffff;fillColor=#3E54E6;"
        elif(actionType == 'generate hypothesis'):
          style += "pd3action=ECGH;strokeColor=#d79b00;fontColor=#ffffff;fillColor=#C93AC9;"
        elif(actionType == 'evaluate/select information'):
          style += "pd3action=ECESI;strokeColor=#d79b00;fontColor=#ffffff;fillColor=#8F4132;"
        elif(actionType == 'execute'):
          style += "pd3action=ECEX;strokeColor=#d79b00;fontColor=#ffffff;fillColor=#2EBAC9;"
      mxCell.set('style', style)
      mxCell.set('vertex', str(1))
      parent = '1'
      for s2, p2, o2 in g.triples((s, pd3.attribution, None)):
        for s3, p3, o3 in g.triples((o2, pd3.id, None)):
          parent = str(o3)
      mxCell.set('parent', parent)
      mxGeometry = ET.SubElement(mxCell, 'mxGeometry')
      for s4, p4, o4 in g.triples((s, pd3.geoBoundingX, None)):
        mxGeometry.set('x', o4)
      for s4, p4, o4 in g.triples((s, pd3.geoBoundingY, None)):
        mxGeometry.set('y', o4)
      for s4, p4, o4 in g.triples((s, pd3.geoBoundingWidth, None)):
        mxGeometry.set('width', o4)
      for s4, p4, o4 in g.triples((s, pd3.geoBoundingHeight, None)):
        mxGeometry.set('height', o4)
      mxGeometry.set('as', 'geometry')
    
    #Flow
    elif( o ==pd3.Flow):
      for s2, p2, o2 in g.triples((s, pd3.layer, None)):
        layer = str(o2)
        if(layer == 'phys'):
          mxCell.set('style', "html=1;verticalAlign=bottom;endArrow=block;rounded=0;pd3layer=phys;pd3type=arrow;fillColor=#d5e8d4;strokeColor=#82b366;fontSize=14;exitX=1;exitY=0.5;entryX=0;entryY=0.5;")
        elif(layer == 'info'):
          mxCell.set('style', "html=1;verticalAlign=bottom;endArrow=block;rounded=0;pd3layer=info;pd3type=arrow;fillColor=#dae8fc;strokeColor=#6c8ebf;fontSize=14;exitX=1;exitY=0.5;entryX=0;entryY=0.5;")
        elif(layer == 'topic'):
          mxCell.set('style', "html=1;verticalAlign=bottom;endArrow=block;rounded=0;pd3layer=info;pd3type=arrow;fillColor=#ffe6cc;strokeColor=#d79b00;endArrow=block;fontSize=14;exitX=1;exitY=0.5;entryX=0;entryY=0.5;")

      parent = '1'
      for s2, p2, o2 in g.triples((s, pd3.source, None)):
        source = o2
        for s3, p3, o3 in g.triples((source, pd3.id, None)):
          mxCell.set('source', o3)
      for s2, p2, o2 in g.triples((s, pd3.target, None)):
        target = o2
        for s3, p3, o3 in g.triples((target, pd3.id, None)):
          mxCell.set('target', o3)
      parent = '1'
      for s3, p3, o3 in g.triples((source, pd3.attribution, None)):
        for s4, p4, o4 in g.triples((target, pd3.attribution, None)):
          if(str(o3)==str(o4)):
            for s5, p5, o5 in g.triples((o3, pd3.id, None)):
              parent = str(o5)
      mxCell.set('parent', parent)
      mxCell.set('edge', "1")
      mxGeometry = ET.SubElement(mxCell, 'mxGeometry')
      mxGeometry.set('relative', '1')
      mxGeometry.set('as', 'geometry')

    #SupFlow
    elif(o == pd3.SupFlow):
      mxGeometry = ET.SubElement(mxCell, 'mxGeometry')
      mxGeometry.set('relative', '1')
      mxGeometry.set('as', 'geometry')
      mxPoint_source = ET.SubElement(mxGeometry, 'mxPoint')
      mxPoint_source.set('as', 'sourcePoint')
      mxPoint_target = ET.SubElement(mxGeometry, 'mxPoint')
      mxPoint_target.set('as', 'targetPoint')

      style = "endArrow=block;rounded=0;endFill=1;html=1;align=left;verticalAlign=middle;pd3type=arc;"
      for s2, p2, o2 in g.triples((s, None, None)):
        if(p2 == pd3.layer):
          layer = str(o2)
          if(layer == 'phys'):
            style += 'pd3layer=phys;fillColor=#d5e8d4;strokeColor=#82b366;'
          elif(layer == 'info'):
            style += 'pd3layer=info;fillColor=#dae8fc;strokeColor=#6c8ebf;'
          elif(layer == 'topic'):
            style += 'pd3layer=topic;fillColor=#ffe6cc;strokeColor=#d79b00;'
        elif(p2 == pd3.arcType):
          arcType = str(o2)
          style += "arctype=" + arcType + ";"
        elif(p2 == pd3.target):
          target = o2
          for s3, p3, o3 in g.triples((target, pd3.id, None)):
            target_id = str(o3)
            mxCell.set('target', target_id)
        # 2022/1/12 add Start Man
        elif(p2 == pd3.source):
          source = o2
          for s3, p3, o3 in g.triples((source, pd3.id, None)):
            source_id = str(o3)
            mxCell.set('source', source_id)
        # 2022/1/12 add End Man
        elif(p2 == pd3.geoSourcePointX):
          geoSourcePointX = str(o2)
          mxPoint_source.set('x', geoSourcePointX)
        elif(p2 == pd3.geoSourcePointY):
          geoSourcePointY = str(o2)
          mxPoint_source.set('y', geoSourcePointY)
        elif(p2 == pd3.geoTargetPointX):
          geoTargetPointX = str(o2)
          mxPoint_target.set('x', geoTargetPointX)
        elif(p2 == pd3.geoTargetPointY):
          geoTargetPointY = str(o2)
          mxPoint_target.set('y', geoTargetPointY)
        elif(p2 == pd3.entryX):
          entryX = str(o2)
          style += "entryX="+entryX+";"
        elif(p2 == pd3.entryY):
          entryY = str(o2)
          style += "entryY="+entryY+";"
      mxCell.set('style', style)
      mxCell.set('edge', '1')
      parent = '1'
      for s3, p3, o3 in g.triples((target, pd3.attribution, None)):
        for s4, p4, o4 in g.triples((o3, pd3.id, None)):
          parent = str(o4)
      mxCell.set('parent', parent)

    #ContainerFlow
    elif(o == pd3.ContainerFlow):
      mxGeometry = ET.SubElement(mxCell, 'mxGeometry')
      mxGeometry.set('relative', '1')
      mxGeometry.set('as', 'geometry')

      style = "edgeStyle=none;rounded=0;orthogonalLoop=0;jettySize=auto;html=1;pd3type=arc;entryX=0;entryY=1;entryDx=0;entryDy=0;endArrow=block;fontSize=14;exitX=0.5;exitY=0;exitDx=0;exitDy=0;"
      for s2, p2, o2 in g.triples((s, None, None)):
        if(p2 == pd3.layer):
          layer = str(o2)
          if(layer == 'phys'):
            style += 'pd3layer=phys;fillColor=#d5e8d4;strokeColor=#82b366;'
          elif(layer == 'info'):
            style += 'pd3layer=info;fillColor=#dae8fc;strokeColor=#6c8ebf;'
          elif(layer == 'topic'):
            style += 'pd3layer=topic;fillColor=#ffe6cc;strokeColor=#d79b00;'
        elif(p2 == pd3.source):
          source = o2
          for s3, p3, o3 in g.triples((source, pd3.id, None)):
            source_id = str(o3)
            mxCell.set('source', source_id)
        elif(p2 == pd3.target):
          target = o2
          for s3, p3, o3 in g.triples((target, pd3.id, None)):
            target_id = str(o3)
            mxCell.set('target', target_id)
        mxCell.set('style', style)
        mxCell.set('edge', '1')
        mxCell.set('parent', '1')
    
    #Container
    elif(o == pd3.Container):
      mxGeometry = ET.SubElement(mxCell, 'mxGeometry')
      mxGeometry.set('as', 'geometry')

      style = "swimlane;pd3type=container;containertype=specialization;fontSize=14;"
      for s2, p2, o2 in g.triples((s, None, None)):
        if(p2 == pd3.layer):
          layer = str(o2)
          if(layer == 'phys'):
            style += 'pd3layer=phys;fillColor=#d5e8d4;strokeColor=#82b366;'
          elif(layer == 'info'):
            style += 'pd3layer=info;fillColor=#dae8fc;strokeColor=#6c8ebf;'
          elif(layer == 'topic'):
            style += 'pd3layer=topic;fillColor=#ffe6cc;strokeColor=#d79b00;'
        elif(p2 == pd3.geoBoundingX):
          mxGeometry.set('x', str(o2))
        elif(p2 == pd3.geoBoundingY):
          mxGeometry.set('y', str(o2))
        elif(p2 == pd3.geoBoundingWidth):
          mxGeometry.set('width', str(o2))
        elif(p2 == pd3.geoBoundingHeight):
          mxGeometry.set('height', str(o2))
      mxCell.set('style', style)
      mxCell.set('vertex', '1')
      mxCell.set('parent', '1')

    # 2022/1/12 add Start Man
    #object
    elif(o == pd3.Object):
      mxGeometry = ET.SubElement(mxCell, 'mxGeometry')

      style = "rounded=0;whiteSpace=wrap;html=1;"
      for s2, p2, o2 in g.triples((s, None, None)):
        if(p2 == pd3.layer):
          layer = str(o2)
          if(layer == 'phys'):
            style += 'pd3layer=phys;strokeColor=#82b366;'
          elif(layer == 'info'):
            style += 'pd3layer=info;strokeColor=#6c8ebf;'
          elif(layer == 'topic'):
            style += 'pd3layer=topic;strokeColor=#d79b00;'
          style += 'pd3type=object;dashed=1;fontSize=14;fillColor=none;strokeWidth=1;'
        elif(p2 == pd3.geoBoundingX):
          mxGeometry.set('x', str(o2))
        elif(p2 == pd3.geoBoundingY):
          mxGeometry.set('y', str(o2))
        elif(p2 == pd3.geoBoundingWidth):
          mxGeometry.set('width', str(o2))
        elif(p2 == pd3.geoBoundingHeight):
          mxGeometry.set('height', str(o2))
      mxCell.set('style', style)
      mxGeometry.set('as', 'geometry')
      parent = '1'
      for s2, p2, o2 in g.triples((s, pd3.attribution, None)):
        for s3, p3, o3 in g.triples((o2, pd3.id, None)):
          parent = str(o3)
      mxCell.set('parent', parent)
      mxCell.set('vertex', '1')
      # 2022/1/12 add End Man

  #ET.dump(mxfile)
  tree = ET.ElementTree(mxfile)
  root1 = tree.getroot()
  document = md.parseString(ET.tostring(root1, encoding='unicode'))
  file = open(xmlpath, 'w', encoding='utf-8')
  # エンコーディング、改行、全体のインデント、子要素の追加インデントを設定しつつファイルへ書き出し
  document.writexml(file, encoding='utf-8', newl='\n', indent='', addindent='  ')
  file.close()

In [8]:
xml_to_ttl('./test2.xml', './test2.ttl')
#ttl_to_xml('./member_test/fuseki_export.ttl', './member_test/fuseki_export.xml')

In [ ]:
xml_to_ttl('/content/熟練者A_20220131.xml', '/content/熟練者A_20220131.ttl')